In [56]:
import re
import math
import warnings
import tqdm
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from sympy import symbols, Eq, solve
from matplotlib.pyplot import MultipleLocator
from sklearn.preprocessing import MinMaxScaler, Binarizer
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
warnings.filterwarnings("ignore")


%matplotlib inline
plt.rcParams['font.sans-serif'] = ['SimHei'] 
plt.rcParams['axes.unicode_minus'] = False 

In [57]:
data1=pd.read_excel('/root/GDUT/表1-患者列表及临床信息.xlsx')
data2=pd.read_excel('/root/GDUT/表2-患者影像信息血肿及水肿的体积及位置.xlsx')
data3=pd.read_excel('/root/GDUT/表3-患者影像信息血肿及水肿的形状及灰度分布.xlsx')
data4=pd.read_excel('/root/GDUT/表4-答案文件.xlsx')
data5=pd.read_excel('2b聚类数据.xlsx')
# data6=pd.read_excel('是否扩张.xlsx')

data1.rename(columns={data1.columns[0]: "ID"}, inplace=True)
data2.rename(columns={data2.columns[0]: "ID"}, inplace=True)

data_f_1=pd.read_excel('附表1-检索表格-流水号vs时间.xlsx')
data_f_time=pd.read_excel('时间点.xlsx')

In [58]:
data6['ID'] = data5['ID'].head(100)
data7 = pd.merge(data5.head(100), data6, how='outer', on='ID')

# 转换独热编码
xueya_list = []
for i in data7['血压']:
    y1 = i.split('/')[0]
    y2 = i.split('/')[1]
    if int(y1) > 139 or int(y1) < 90:
        status_i = 0
    elif int(y2) > 89 or int(y2) < 60:
        status_i = 0
    else:
        status_i = 1
    xueya_list.append(status_i)
data7['血压'] = np.array(xueya_list)
data7['性别'] = data7['性别'].map({'男': np.array([0, 1]), '女': np.array([1, 0])})
x = data7['年龄'].values
scaler = MinMaxScaler()
x_normalized = scaler.fit_transform(x.reshape(-1, 1))
binarizer = Binarizer(threshold=0.5)
x_binarized = binarizer.fit_transform(x_normalized)
data7['年龄'] = x_binarized

In [59]:
data7['年龄'].values

array([0., 0., 1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0.,
       1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 1.,
       1., 0., 0., 1., 1., 1., 0., 1., 1., 0., 0., 0., 1., 1., 1., 1., 0.,
       1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0., 1.])

In [62]:
data7

,ID,年龄,性别,脑出血前mRS评分,高血压病史,卒中病史,糖尿病史,房颤史,冠心病史,吸烟史,饮酒史,血压,是否扩张
0,sub001,0.0,"[1, 0]",0,0,0,0,0,0,0,0,0,0
1,sub002,0.0,"[0, 1]",0,1,0,0,0,0,0,0,0,0
2,sub003,1.0,"[0, 1]",0,1,0,0,0,0,0,0,0,1
3,sub004,1.0,"[0, 1]",2,1,1,0,0,0,0,0,0,0
4,sub005,0.0,"[0, 1]",0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,sub096,1.0,"[1, 0]",0,0,0,0,0,0,0,0,0,0
96,sub097,1.0,"[1, 0]",0,1,0,0,0,0,0,0,0,0
97,sub098,1.0,"[0, 1]",0,1,1,1,1,1,1,0,0,0
98,sub099,0.0,"[0, 1]",0,1,0,0,0,0,0,0,0,1
